In [269]:
from requests import get
from bs4 import BeautifulSoup

In [273]:
#creating loop to create urls for 50 movies at a time

#initializing iterator variable (pages increment in form of 'start=1', then 'start=51', then 'start=101'......)
z = 1

#creating empty list for urls to be stored in
urls = []

#while the number of movies is below 4002 (arbitrary #)
while z < 4002:
    
    #generating url using format to insert the incrementing variable on each loop
    url = 'https://www.imdb.com/search/title/?title_type=feature&moviemeter=,4000&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(z)
    
    #appending the url to the list of urls
    urls.append(url)
    
    #incrementing iterator variable so it can go to the next page (there are 50 movies listed per page)
    z += 50


In [274]:
#for loop to get all the html objects of each array into a soup object

#creating empty soup item to store the generated html files in
html_soups = BeautifulSoup()

#iterating through the url pages
for url in urls:
    
    #creating response to use in BeautifulSoup function
    response = get(url)
    
    #using the BeautifulSoup function on each response, appending the resulting soup object into the empty 'html_soups'
    html_soups.append(BeautifulSoup(response.text, 'html.parser'))

In [275]:
#creating containers for each movie, seperating at 'lister-item mode-advanced' on html doc
m_containers = html_soups.find_all('div', class_='lister-item mode-advanced')

#printing to make sure the results are what we expect; if you go to the URL, there are only 2758 items on the list created, all are here
print(type(m_containers))
print(len(m_containers))

<class 'bs4.element.ResultSet'>
2758


In [305]:
#creating empty arrays to store individual movie info in
titles = []
years = []
imdb_ratings = []
summs = []
mpaa_rates = []

In [306]:
#for loop to parse through each movie in m_containers and get the info
for container in m_containers:
    
    #finding title of movie through its location in html doc
    title = container.h3.a.text
    
    #appending title to empty title list
    titles.append(title)

    
    #finding year of movie through its location in html doc
    year = container.h3.find('span', class_='lister-item-year').text
    
    #appending year to empty year list
    years.append(year)

    
    #if function created to deal with any movies that dont have ratings
   
    #if there is a rating found in the html
    if container.strong is not None:
        
        #imdb rating stored in i_rate
        i_rate = float(container.strong.text)
        
        #value added to imdb ratings list
        imdb_ratings.append(i_rate)
        
    #if there is NO imdb rating:
    else:
        
        #set it to 0
        imdb_ratings.append(0)

        
    #finding mpaa rating (G/PG/R etc.) in html
    rating = container.p.span.text
    
    #appending mpaa rating to mpaa_rates list
    mpaa_rates.append(rating)
    

    #finding summary of movie in html
    summary = container.find_all('p', class_='text-muted')[1].text.strip('\n   ')
    
    #appending summary to summs list
    summs.append(summary)

In [307]:
#checking if all lengths match up
print(len(titles))
print(len(summs))
print(len(years))
print(len(imdb_ratings))
print(len(mpaa_rates))

2758
2758
2758
2758
2758


To do tomorrow:
1. find a way to get the documents into json format to be transferred to mongodb db

2. Begin process of creating functionality - research how to do the things we want to do

In [ ]:
test='test'

In [315]:
import pandas as pd
df = pd.DataFrame({'movie': titles,
'year': years,
'imdb': imdb_ratings,
'summary': summs,
'mpaa_rates': mpaa_rates
})

#removing movies with 0's as they arent out yet...
df = df[df.imdb != 0]